# What this file does

Research question: Do SVI ZIP codes line up with ZIP codes with high death rates in CT? 

Answer: Not really? Weak correlation. 60% of the top 60 ZIP codes in terms of death rate are not SVI ZIP codes. Put another way: Just under half of SVI ZIP codes appear in the top-60 list of ZIP codes sorted by death rate per capita. Preliminary viz: https://public.flourish.studio/visualisation/5560351/

Sources: CT DH Provisional Death data (through Nov 2020), CT DPH's list of SVI zip codes, Population by ZIP code from Cubit (https://www.connecticut-demographics.com/zip_codes_by_population; <b>Note to me: Spot check this data's quality</b>), CT Data Collaborative's ZIP code to town converter (https://github.com/CT-Data-Collaborative/ct-zip-to-town/blob/master/data/zip2town.csv)

Methodology notes: Death rates factor out nursing home, hospice deaths where death certificates indicate that the person died there. The analysis considers the ZIP code that the person lived in, not where they died. Data do not include deaths from second wave.

Index:
- 1. Wrangling death data 
- 2. Cleaning ZIP code data 
- 3. Merging with ZIP code population 
- 4. Comparing with SVI list
- 5. Mapping back to towns
- 6. Pearson correlation coefficent

In [226]:
#tools
import pandas as pd

# 1. Wrangling death data

Data dictionary in separate excel file.

In [227]:
#read in file; provisional data through Nov 2020
df = pd.read_csv('deaths_deid_2020prov_v20210225.csv', encoding='latin-1', dtype={'DOD': object})
df['Date of Death (Fixed)'] = pd.to_datetime(df['DOD'] + df['DTHYR'].astype(str), format = '%m%d%Y')

/Users/kasturipananjady/.pyenv/versions/3.7.1/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (22,53,54,55) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [228]:
#filtering for COVID
df_covid = df[(df['PCAUSE']=='U071')|(df['PCAUSE']=='U072')|(df['COD1']=='U071')|(df['COD1']=='U072')|(df['COD2']=='U071')|(df['COD2']=='U072')|(df['COD3']=='U071')|(df['COD3']=='U072')|(df['COD4']=='U071')|(df['COD4']=='U072')|(df['COD5']=='U071')|(df['COD5']=='U072')|(df['COD6']=='U071')|(df['COD6']=='U072')|(df['COD7']=='U071')|(df['COD7']=='U072')|(df['COD8']=='U071')|(df['COD8']=='U072')|(df['COD9']=='U071')|(df['COD9']=='U072')|(df['COD10']=='U071')|(df['COD10']=='U072')|(df['COD11']=='U071')|(df['COD11']=='U072')|(df['COD12']=='U071')|(df['COD12']=='U072')|(df['COD13']=='U071')|(df['COD13']=='U072')|(df['COD14']=='U071')|(df['COD14']=='U072')|(df['COD15']=='U071')|(df['COD15']=='U072')]

/Users/kasturipananjady/.pyenv/versions/3.7.1/lib/python3.7/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/Users/kasturipananjady/.pyenv/versions/3.7.1/lib/python3.7/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/Users/kasturipananjady/.pyenv/versions/3.7.1/lib/python3.7/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/Users/kasturipananjady/.pyenv/versions/3.7.1/lib/python3.7/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perfo

In [229]:
#sanity check; yes, lines up with end of Nov COVID nos, roughly 
len(df_covid)

5124

In [230]:
#take out nursing homes
df_covid_not_nursing_homes = df_covid[df_covid['POD_TYPE_label']!='Nursing Home']

In [231]:
#take out hospice
df_covid_not_nursing_homes_or_hospice = df_covid_not_nursing_homes[df_covid_not_nursing_homes['POD_TYPE_label']!='Hospice Facility']

In [232]:
#save deaths by zipcode
df_zipcodes_deaths = pd.DataFrame(df_covid_not_nursing_homes_or_hospice['RES_ZIP'].value_counts()).reset_index()
df_zipcodes_deaths = df_zipcodes_deaths.rename(columns = {
    'index':'ZIPcode',
    'RES_ZIP':'Deaths'
})

# 2. Cleaning up ZIP codes

In [233]:
#need to clean; two problems — inconsistent 0s in front and decimal points
df_zipcodes_deaths['ZIPcode'].unique()[::15]

array(['06902', 6514.0, '06112', 6605.0, 6850.0, 6385.0, 6120.0, '06451',
       6484.0, 6706.0, '06032', 6320.0, 6905.0, '06401', '06042', 6607.0,
       6098.0, '06375', 6419.0, '06482', '06019', '06497', '06050',
       6422.0, '06719', '13205', '10452', '11210', 29582.0, 6250, '10475',
       '12538'], dtype=object)

In [234]:
#function to clean ZIP codes; remove zero at the top if needed
def remove_zero(x):
    if (str(x)[0]) == '0':
        return str(x)[1::]
    else:
        return str(x)
    

In [235]:
#function to clean ZIP codes; remove decimal places if needed
def remove_decimals(x):
    if (str(x)[-2::]) == '.0':
        return str(x)[0:-2]
    else:
        return str(x) 

In [236]:
#cleaning zipcodes
df_zipcodes_deaths['ZIPcode'] = df_zipcodes_deaths['ZIPcode'].apply(lambda x: remove_zero(x))
df_zipcodes_deaths['ZIPcode'] = df_zipcodes_deaths['ZIPcode'].apply(lambda x: remove_decimals(x))

In [237]:
#cleaned?
df_zipcodes_deaths_cleaned = pd.DataFrame(df_zipcodes_deaths.groupby(by='ZIPcode')['Deaths'].sum()).reset_index().sort_values(by='Deaths', ascending=False)

# 3. Merging w population data for rate calculation

In [238]:
#read in file
df_zipcodes_population = pd.read_csv('zipcodes_population.csv')

In [239]:
#set datatypes
df_zipcodes_population['Population'] = df_zipcodes_population['Population'].str.replace(',','').astype(int)
df_zipcodes_population['ZIPcode'] = df_zipcodes_population['Zip Code'].astype(str)

In [240]:
#merge, outer join (don't want to lose any death data here)
df_final = df_zipcodes_deaths_cleaned.merge(df_zipcodes_population, left_on='ZIPcode', right_on='ZIPcode', how='outer')

In [241]:
#rate calculation, sort, isolation of the top 50
df_final['Deaths_per_100000'] = 100000*df_final['Deaths']/df_final['Population']
df_final_top_50 = df_final.sort_values(by='Deaths_per_100000', ascending=False).head(50)

# 4. Comparing with SVI list

In [242]:
#read in the file
df_svi = pd.read_csv('zipcodes.csv')

In [243]:
#add column, set datatype
df_svi['SVI']='SVI'
df_svi['Zipcode'] = df_svi['Zipcode'].apply(lambda x: remove_decimals(x))

In [244]:
#merge
df_death_rate_vs_SVI_top_50 = df_final_top_50.merge(df_svi, left_on='ZIPcode', right_on='Zipcode', how='left')

In [245]:
#all the SVI ZIP codes merge wheeee!
df_final_final =df_final.merge(df_svi, left_on='ZIPcode', right_on='Zipcode', how='left')

In [246]:
#the majority of top-50 death rate zipcodes are not SVI zipcodes by the state's reckoning 
df_death_rate_vs_SVI['SVI'].value_counts(dropna=False)

NaN    31
SVI    19
Name: SVI, dtype: int64

# 5. Mapping back to towns

In [249]:
#read in file, set data type
df_zips_to_towns = pd.read_csv('zipcode_to_town.csv')
df_zips_to_towns['zip5'] = df_zips_to_towns['zip5'].astype(str)

In [250]:
#merging so as not to lose any death data
df_done = df_zips_to_towns.merge(df_final_final, left_on='zip5', right_on='ZIPcode', how='right')

In [253]:
#for flourish viz
df_done.to_clipboard()

In [260]:
#discard the first Norwalk zipcode; seems very small, population is ridiculously tiny
df_to_save = df_done[['town', 'county','ZIPcode','Deaths','Population','Deaths_per_100000','SVI']].sort_values(by='Deaths_per_100000', ascending=False).drop(203)

In [263]:
#top 60 
df_to_save['SVI'].head(60).value_counts(dropna=False)

NaN    36
SVI    24
Name: SVI, dtype: int64

In [264]:
#top 50
df_to_save['SVI'].head(50).value_counts(dropna=False)

NaN    31
SVI    19
Name: SVI, dtype: int64

In [267]:
#save
df_to_save.to_csv('SVI_vs_COVID_death_rate_zip.csv', index=False)

# 6. Pearson correlation coefficient

In [269]:
#isolate variables that we need
df_for_corr = df_to_save[['Deaths_per_100000','SVI']]

In [273]:
#convert categorical to numeric
def convert_categorical_to_numeric(x):
    if x =='SVI':
        return 1
    else: 
        return 0 
    
df_for_corr['SVI_numeric'] = df_for_corr['SVI'].apply(lambda x:convert_categorical_to_numeric(x) )

In [278]:
#weak correlation
df_for_corr.corr()

,Deaths_per_100000,SVI_numeric
Deaths_per_100000,1.000000,0.249183
SVI_numeric,0.249183,1.000000
